In [41]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

import warnings
warnings.filterwarnings('ignore')

# Zadanie
Proszę podzielić dane na train test. Nstępnie wykonać walidację krzyzową na train i zewaluować na test:
* ElasticNet	
* Lasso	
* Ridge	
* LR	
* SVR	
* RFR	

Proszę nie zmieniać preprocesingu danych

https://archive.ics.uci.edu/ml/datasets/automobile

https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data

In [3]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"
names = ['symboling','normalized-losses','make','fuel-type', 'aspiration', 'num-of-doors', 'body-style', 'drive-wheels', 'engine-location', 'wheel-base', 'length', 'width', 'height', 'curb-weight', 'engine-type', 'num-of-cylinders', 'engine-size', 'fuel-system', 'bore', 'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']


In [4]:
data = pd.read_csv(url, names=names, delimiter=",", na_values=["?"])
print(data.shape)

(205, 26)


In [5]:
data.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0


# Zadanie
Proszę usunąć kolumny, które nie są numeyczne.

In [6]:
drop_list = ['symboling','make','make','fuel-type', 'aspiration', 'num-of-doors', 
             'body-style', 'drive-wheels', 'engine-location','engine-type','num-of-cylinders','fuel-system']

In [7]:
data=data.drop(drop_list, axis=1)
data.head()

,normalized-losses,wheel-base,length,width,height,curb-weight,engine-size,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,NaN,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,NaN,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,NaN,94.5,171.2,65.5,52.4,2823,152,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,164.0,99.8,176.6,66.2,54.3,2337,109,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,164.0,99.4,176.6,66.4,54.3,2824,136,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0


# Zadanie
Prosze usunąć brakujące dane.

In [8]:
null_counts = data.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))

Number of null values in each column:
normalized-losses    41
wheel-base            0
length                0
width                 0
height                0
curb-weight           0
engine-size           0
bore                  4
stroke                4
compression-ratio     0
horsepower            2
peak-rpm              2
city-mpg              0
highway-mpg           0
price                 4
dtype: int64


In [9]:
data=data.dropna()
data.head()

,normalized-losses,wheel-base,length,width,height,curb-weight,engine-size,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
3,164.0,99.8,176.6,66.2,54.3,2337,109,3.19,3.4,10.0,102.0,5500.0,24,30,13950.0
4,164.0,99.4,176.6,66.4,54.3,2824,136,3.19,3.4,8.0,115.0,5500.0,18,22,17450.0
6,158.0,105.8,192.7,71.4,55.7,2844,136,3.19,3.4,8.5,110.0,5500.0,19,25,17710.0
8,158.0,105.8,192.7,71.4,55.9,3086,131,3.13,3.4,8.3,140.0,5500.0,17,20,23875.0
10,192.0,101.2,176.8,64.8,54.3,2395,108,3.50,2.8,8.8,101.0,5800.0,23,29,16430.0


# Zadanie
Proszę nauczyć wszytkie poznane modele regresji.
Który z nich działa najlepiej?

In [10]:
auto_target = data["price"]
auto_data = data.drop(["price"],axis=1)

In [11]:
auto_data.head()

,normalized-losses,wheel-base,length,width,height,curb-weight,engine-size,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg
3,164.0,99.8,176.6,66.2,54.3,2337,109,3.19,3.4,10.0,102.0,5500.0,24,30
4,164.0,99.4,176.6,66.4,54.3,2824,136,3.19,3.4,8.0,115.0,5500.0,18,22
6,158.0,105.8,192.7,71.4,55.7,2844,136,3.19,3.4,8.5,110.0,5500.0,19,25
8,158.0,105.8,192.7,71.4,55.9,3086,131,3.13,3.4,8.3,140.0,5500.0,17,20
10,192.0,101.2,176.8,64.8,54.3,2395,108,3.50,2.8,8.8,101.0,5800.0,23,29


In [39]:
auto_target.head()

3     13950.0
4     17450.0
6     17710.0
8     23875.0
10    16430.0
Name: price, dtype: float64

In [33]:
y=auto_target
X=auto_data
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
kfold = KFold(10)

In [38]:
grid1 = GridSearchCV(make_pipeline(PolynomialFeatures(1), ElasticNet(1)),
                    cv=5,
                    param_grid={'polynomialfeatures__degree':[1,2,3,4,5],
                                'elasticnet__alpha' : [0.1,1,10,100]})

grid1.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('polynomialfeatures',
                                        PolynomialFeatures(degree=1)),
                                       ('elasticnet', ElasticNet(alpha=1))]),
             param_grid={'elasticnet__alpha': [0.1, 1, 10, 100],
                         'polynomialfeatures__degree': [1, 2, 3, 4, 5]})

In [ ]:
grid2 = GridSearchCV(make_pipeline(PolynomialFeatures(1), Lasso(1)),
                    cv=5,
                    param_grid={'polynomialfeatures__degree':[1,2,3,4,5],
                                'lasso__alpha' : [0.1,1,10,100]})

grid2.fit(X_train,y_train)

In [ ]:
grid3 = GridSearchCV(make_pipeline(PolynomialFeatures(1), Ridge(1)),
                    cv=5,
                    param_grid={'polynomialfeatures__degree':[1,2,3,4,5],
                                'ridge__alpha' : [0.1,1,10,100]})

grid3.fit(X_train,y_train)

In [ ]:
grid4 = GridSearchCV(make_pipeline(PolynomialFeatures(1), LinearRegression()),
                    cv=5,
                    param_grid={'polynomialfeatures__degree':[1,2,3,4,5]})

grid4.fit(X_train,y_train)

In [ ]:
grid5 = GridSearchCV(SVR(kernel='rbf',gamma=0.1),
                    cv=5,
                    param_grid={'C':[0.00001,0.0001,0.001,0.01,0.1,1,10,100,1000,10000],
                                'gamma' : [0.0001,0.001,0.01,0.1,1,10,100,1000,10000]})

grid5.fit(X_train,y_train)

In [ ]:
grid6 = GridSearchCV(RandomForestRegressor(),
                    cv=5,
                    param_grid={'max_depth':[1,2,3,4,5,6,7,8],
                                'n_estimators' : [1,2,3,4,5,6,7,8]})

grid6.fit(X_train,y_train)